# Image Morphing
_by Edoardo Maines (mat. 232226) and Davide Modolo (mat. 229297)_

In [ ]:
%pip install numpy opencv-contrib-python Pillow

## Import python libs + our "resize"
The morph function of opencv needs two images of the exact same size, so we wrote our own library to resize them without cutting pixels (dragging + padding).

In [ ]:
import numpy as np
from PIL import Image
import cv2 # for facial rec
import math # to round
import resize # our lib to resize
from IPython.display import clear_output #to write animations progress

## Fading
The easiest morphing approach is a simple fade between the two images. We start with the first having 100% alpha value and the second with 0%, then at each iteration we decrease the first by an arbitrary 2% and we increase the second by 2%.

In [ ]:
# FADING
def fading(img1,img2):
    for a in range(0, 102, 2):
        # calculate current alpha step
        alpha_value = a/100
        # compute the result image
        img3 = Image.blend(img1, img2, alpha_value)
        #show the frame
        cv2.imshow("Fade", np.array(img3))
        clear_output(wait=True)
        print('Alpha value: ', round(alpha_value,2))
        cv2.waitKey(25)
    cv2.waitKey()

## Morphing Function
OpenCV has a warp function, that takes 3 points of the first image, 2 points of the second image and the result is a warped second image based on the first. In this function, we compute each individual step, getting the animation as result.
The idea has been taken by this youtube video: https://www.youtube.com/watch?v=56rxZGU7JxQ

In [ ]:
def morphImage(img1, img1points, img2, img2points):
    n_points = 3
    steps = 100
    frame = 1.0/steps
    # create empty lists
    pts11 = np.zeros((n_points, 2), np.float32)
    pts22 = np.zeros((n_points, 2), np.float32)
    pts1 = np.zeros((n_points, 2), np.float32)
    pts2 = np.zeros((n_points, 2), np.float32)
    # fill the list of points for each image
    for i in range(n_points):
        pts1[i] = img1points[i]
        pts2[i] = img2points[i]
    # for each frame
    for i in range(steps):
        # for each of 3 points
        for j in range(n_points):
            # some math to calclate the distance
            disx = (pts1[j, 0] - pts2[j, 0])*-1
            disy = (pts1[j, 1] - pts2[j, 1])*-1
            
            # calculate movements of each point of each images
            movex1 = (disx/steps) * (i+1)
            movey1 = (disy/steps) * (i+1)
            
            movex2 = disx-movex1
            movey2 = disy-movey1
            
            # perform the "movement"
            pts11[j, 0] = pts1[j, 0] + movex1
            pts11[j, 1] = pts1[j, 1] + movey1

            pts22[j, 0] = pts2[j, 0] - movex2
            pts22[j, 1] = pts2[j, 1] - movey2
            
        # after computing the movement of each of the three points we get the matrix
        mat1 = cv2.getAffineTransform(pts1, pts11)
        mat2 = cv2.getAffineTransform(pts2, pts22)
        # we compute the "warping step"
        dst1 = cv2.warpAffine(
            img1, mat1, (img1.shape[1], img1.shape[0]), None, None, cv2.BORDER_REPLICATE)

        dst2 = cv2.warpAffine(
            img2, mat2, (img1.shape[1], img1.shape[0]), None, None, cv2.BORDER_REPLICATE)
        # we sum up the result of the two pictures
        result = cv2.addWeighted(dst1, 1-(frame*(i)), dst2, frame*(i+1), 0)
        # we show the frame
        cv2.imshow("Morph", result)
        clear_output(wait=True)
        print('Morphing Steps: ', i+1)
        if(i == steps-1):
            cv2.waitKey()
        else:
            cv2.waitKey(25)

## Morphing by Point-picking
We ask the user to pick three points in each image (with coherent order) and then we call the morphing function.

In [ ]:
def picPoint(event, x, y, flags, param):
    global point
    if event == cv2.EVENT_LBUTTONDOWN:
        # when mouse left clicked, save its coordinates
        point = (float(x), float(y))
        cv2.destroyAllWindows()

def manualPick(img1, img2):
    n_points = 3
    # initialize variables to save the picked points
    img1points = []
    img2points = []
    for i in range(n_points):
        # show first image
        cv2.imshow('First Image', img1)
        # recall the function when mouse clicked
        cv2.setMouseCallback('First Image', picPoint)
        cv2.waitKey(0)
        # append the clicked point to the list
        img1points.append(point)
        # repeat for image 2
        cv2.imshow('Second Image', img2)
        cv2.setMouseCallback('Second Image', picPoint)
        cv2.waitKey(0)
        img2points.append(point)
    # return picked points
    return img1points, img2points

## Autopicking points
If the two images are not faces, we thought about a way to pick 3 points according to some "similarity" between them.
First we subdivide the first image in a grid (in this case 10x10), then we check for each "cell" the most similar point in the second image.
We save every max-similarity point in a list, only if
- there is not another similarity point in the same region 
- the points in the two images are not too far between
- only if under 100%, because otherwise il will probably be in the start/end/top/bottom padding

If no similarity point is found, we arbitrarely assign them.

In [ ]:
# function to calculate the distance between two points
def distance(pt1, pt2):
    first_diff = (pt2[0]-pt1[0])**2
    second_diff = (pt2[1]-pt1[1])**2
    return round(math.sqrt(first_diff+second_diff))

def autoPick(img1, img2):
    # decide in how many rows and columns we want to subdivide our first image
    k_rows = 10
    k_cols = 10
    # calculate each "cell" width and height
    k_width = round(img1.shape[1]/k_rows)
    k_height = round(img1.shape[0]/k_cols)

    interesting_points = []

    for i in range(k_rows):
        for j in range(k_cols):
            # crop the current cell from the first image
            cell = img1[j*k_height:(j+1)*k_height, i*k_width:(i+1)*k_width]
            # find the maximum similarity with opencv
            result = cv2.matchTemplate(img2, cell, cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            # calculate the center of the current cell relative to the full image1
            cell_center = (round(k_width*(1+2*i)/2), round(k_height*(1+2*j)/2))

            # first we check if the two points are a bit close
            # and if the similarity is not 100% (otherwise it could be just padding)
            distance_threshold = img1.shape[1]/5*2 # it's like an hyperparameter
            if(distance(cell_center, max_loc) < (distance_threshold) and max_val < 100):
                add = True
                # now we check if there already is another close point in the list of
                # chosen possible points
                threshold_new_point = img1.shape[1]/2 # another hyperparameter
                for point in interesting_points:
                    if distance(point[1], cell_center) < threshold_new_point:
                        add = False\
                # then, if there is not, we add the center point to the possible points to
                # select for morphing
                if add:
                    interesting_points.append(
                        (round(max_val, 3), cell_center, max_loc))
                # resulting in a list (their_similarity, point_img1, point_img2)
    
    # we then sort by similarity reversed to have first the points with higher similarity
    interesting_points.sort(reverse=True)
    
    n_points = 3
    # if for any reason there aren't at least three points in the list, we arbitrarely chose them
    if(len(interesting_points) < (n_points)):
        print('Not enough points')
        interesting_points = [(1, (0, 0), (10, 10)), (1, (img1.shape[0]/2, img1.shape[0]/2),
                                                      (img2.shape[0]/2, img2.shape[0]/2)), (1, (300, 20), (300, 100))]
    # we pick the first three points of the list to morph, ordered by the first points
    interesting_points = interesting_points[:n_points]
    interesting_points.sort(key=lambda x: x[1][1])
    # we create the list of points in the format our morphing function wants
    img1points = np.zeros((n_points, 2), np.float32)
    img2points = np.zeros((n_points, 2), np.float32)
    for i in range(n_points):
        img1points[i] = interesting_points[i][1]
        img2points[i] = interesting_points[i][2]
    # return selected points   
    return img1points,img2points

## Face Detection
Disclaimer: **dlib** is hard to install correctly, this is why it doesn't appear in the first block (installing dependencies)

We arbitrarely decided to take points 38 (left eye), 45 (right eye) and 67 (lower mouth lip) of the two recognized faces.

<img src="resources/landmarks.png" style="zoom:25%" border="0px" position="center">

In [ ]:
import dlib

def facePoints(img1, img2):
    img1 = np.array(img1)
    img2 = np.array(img2)
    # load the face detector
    detector = dlib.get_frontal_face_detector()
    landmark_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    # initialize lists of points we want to use for morphing
    points1 = []
    points2 = []
    # convert to gray and detect
    gray = cv2.cvtColor(src=img1, code=cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    # if in the fist image there is no face, call the autopick function
    if(len(faces) < 1):
        return autoPick(img1, img2)
    
    for _, d in enumerate(faces):
        # select points 38, 45 and 67 from the first picture
        landmarks = landmark_detector(img1, d)
        x = landmarks.part(38).x
        y = landmarks.part(38).y
        points1.append((x,y))
        x = landmarks.part(45).x
        y = landmarks.part(45).y
        points1.append((x,y))
        x = landmarks.part(67).x
        y = landmarks.part(67).y
        points1.append((x,y))
        break
    # convert to gray and detect
    gray = cv2.cvtColor(src=img2, code=cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    # if in the second image there is no face, call the autopick function
    if(len(faces) < 1):
        return autoPick(img1, img2)
    
    for k, d in enumerate(faces):
        # select points 38, 45 and 67 from the second picture
        landmarks = landmark_detector(img2, d)
        x = landmarks.part(38).x
        y = landmarks.part(38).y
        points2.append((x,y))
        x = landmarks.part(45).x
        y = landmarks.part(45).y
        points2.append((x,y))
        x = landmarks.part(67).x
        y = landmarks.part(67).y
        points2.append((x,y))
        break
        
    return points1, points2

In [ ]:
img1 = Image.open("images/pic1.jpg")
img2 = Image.open("images/pic2.jpg")
img1, img2 = resize.normalizeImages(img1, img2)

In [ ]:
# FADING - requires two PIL images
fading(img1, img2)

In [ ]:
# convert to numpy for the functions below
img1 = np.array(img1)
img2 = np.array(img2)

In [ ]:
# AUTO PICK
img1points, img2points = autoPick(img1, img2)
morphImage(img1, img1points, img2, img2points)

In [ ]:
# MANUAL PICK
img1points, img2points = manualPick(img1, img2)
morphImage(img1, img1points, img2, img2points)

In [ ]:
# FACE REC
img1points, img2points = facePoints(img1, img2)
morphImage(img1, img1points, img2, img2points)